# Profitable App Profiles for the App Store and Google Play Markets

I work for a company that makes free Android and iOS mobile apps.  Our revenue comes mostly from the in-app ads, so the number of users we have is the most importnant metric.

The goal is to analyze data to help our developers understand what type of apps are likely to attract the most users.

Let's start exploring the data by first opening the two datasets and parsing out the data and header.

In [1]:
from csv import reader

#Google Play dataset
opened_file = open('googleplaystore.csv', encoding='utf8')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android_data = android[1:]

#Apple Store dataset
opened_file = open('AppleStore.csv', encoding='utf8')
read_file = reader(opened_file)
apple = list(read_file)
apple_header = apple[0]
apple_data = apple[1:]

Now we'll write a function called `explore_data()` to display a slice of the data and to count the number of rows and columns.  We'll display the first 5 rows amd the header for each dataset.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
print('Google Play Store')
print(android_header)
print('\n')
explore_data(android_data, 0, 5, True)
print('\n')
print('Apple App Store')
print(apple_header)
print('\n')
explore_data(apple_data, 0, 5, True)


Google Play Store
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+',

# Cleaning the Data


## Removing Incorrect Rows
Now we need to clean the data. First we detect if there is any information missing from any of rows.  We'll do this by making sure every row in the data is the same length as the header.  This will only detect a problem if a piece of data is missing, but not if data gets swapped around.

In [4]:
def del_missing_row(header, data):
    header_length = len(header)
    for row in data:
        if len(row) != header_length:
            print(row)
            print('row ' + str(data.index(row)) + ' is wrong')
            del data[data.index(row)]

In [5]:
del_missing_row(android_header, android_data)
del_missing_row(apple_header, apple_data)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
row 10472 is wrong


## Removing Duplicate Entries
The next step in cleaning up the data is to find and remove any duplicates.  We do this by iterating through the dataset and checking each name against a unique apps list.  If the name is not already found in the list, then it will be added.  If the name is already in the unique apps list, it will instead be added to the duplicate apps list.

In [6]:
def find_duplicates(dataset):
    duplicate_apps = []
    unique_apps = []
    
    for app in dataset:
        name = app[0]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
            
    print('The number of duplicate apps is ' + str(len(duplicate_apps)))

In [7]:
find_duplicates(android_data)
find_duplicates(apple_data)

The number of duplicate apps is 1181
The number of duplicate apps is 0


There are 1181 duplicate apps in the Google Play Store.  Let's look at an example of what these duplicate entries look like.

In [8]:
for app in android_data:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Using Instagram as an example, the duplicates seem to be data on the same app collected at a different time.  The main difference in the duplicates is the total number of reviews the app has.  We want the most recent entry so we will only include the row that has the highest number of reviews and then get rid of the rest.  We end up with 9659 unique Google Play Store apps.

To do this we'll create a dictionary where each key:value pair is a unique app name:highest number of reviews for that app.

In [9]:
#Create a dictionary with only unique app names and the highest review
reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

Now we'll use that dictionary to recreate the dataset without any duplicates.  First we'll initialize two empty lists: `android_no_dupes` and `already_added`.  Then we loop through the Android dataset and add the current app to both lists if two criteria are met:
* The number of reviews of the app matches what's in the above dictionary we just made.
* The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for `reviews_max[name] == n_reviews`, we'll still end up with duplicate entries for some apps.

In [10]:
android_no_dupes = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if n_reviews == reviews_max[name] and name not in already_added:
        android_no_dupes.append(app)
        already_added.append(name)

explore_data(android_no_dupes, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


We end up with 9659 rows, which is exactly what we expected.

## Removing Non-English Apps
We only build apps that are directed towards an English-speaking audience, so we won't need any apps that contain non-English characters in its name.  We can do this by taking advantage of the ASCII system and the `ord()` function.  The most common characters used in English all fall in an ASCII range of 0 to 127.  Thus we should remove any apps with names that has a character outside of this range.  However, we have to be careful because there are certain characters like the trademark symbol or emojis that do not fall between 0 and 127 ASCII that could be part of the name and still be English.  Thus we will only get rid of apps if more than 50% of the characters in the name are not in between 0 and 127 in ASCII

In [11]:
def english_or_not(string):
    nonEnglish = 0
    
    for char in string:
        if ord(char) > 127:
            nonEnglish += 1
            
    pct_nonEnglish = float(nonEnglish)/float(len(string))
    if pct_nonEnglish > .50:
        return False
    else:
        return True

Now we'll remove all the non-English apps in both datasets and store the reduced datasets in `android_clean` and `apple_clean`.

In [12]:
android_clean = []
apple_clean = []

for app in android_no_dupes:
    name = app[0]
    if english_or_not(name):
        android_clean.append(app)
        
for app in apple_data:
    name = app[2]
    if english_or_not(name):
        apple_clean.append(app)

print('For Android')
explore_data(android_clean, 0, 0, True)
print('\n')
print('For Apple')
explore_data(apple_clean, 0, 0, True)

For Android
Number of rows: 9635
Number of columns: 13


For Apple
Number of rows: 6239
Number of columns: 17


# Filtering For Free Apps
Since our company only deals with free apps, the last step in the data cleaning process is to filter out apps that cost money to download.  We will store the free apps in `android_free` and `apple_free`, respectively.

In [14]:
android_free = []
apple_free = []

for app in android_clean:
    price = app[7]
    if price == '0.0' or price == '0':
        android_free.append(app)
        
for app in apple_clean:
    price = app[5]
    if price == '0.0' or price == '0':
        apple_free.append(app)
        
print('For Android')
explore_data(android_free, 0, 0, True)
print('\n')
print('For Apple')
explore_data(apple_free, 0, 0, True)

For Android
Number of rows: 8882
Number of columns: 13


For Apple
Number of rows: 3265
Number of columns: 17


# Analysis

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we build an iOS version and add it to the Apple Store.

Because our end goal is to add the app on both the Google Play and the Apple Store, we need to find apps that are successful on both markets.  Let's begin by getting a sense of the most common app genres in each market.  We'll do this by creating frequency tables.


First, we'll create a dictionary that takes in a `dataset` and an `index` that will count the number of instances in the given column index of each value.  We'll use `index=9` and `index=12` for the android and apple datasets, respetively, to grab the genres.

In [ ]:
def freq_table(